In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

BASE_DIR = 'UTKFace'
age_labels = []
image_paths = []

image_filenames = os.listdir(BASE_DIR)
np.random.shuffle(image_filenames)

for image in tqdm(image_filenames):
    image_path = os.path.join(BASE_DIR, image)
    img_components = image.split('_')
    age_label = int(img_components[0])
    age_labels.append(age_label)
    image_paths.append(image_path)

df = pd.DataFrame()
df['image_path'], df['age'] = image_paths, age_labels

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1)(x)  


model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mean_squared_error', metrics=['mae'])


train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    x_col='image_path',
    y_col='age',
    target_size=(200, 200),
    batch_size=32,
    color_mode='rgb',  
    class_mode='raw',
    subset='training'
)

validation_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    x_col='image_path',
    y_col='age',
    target_size=(200, 200),
    batch_size=32,
    color_mode='rgb',
    class_mode='raw',
    subset='validation'
)


history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

loss, mae = model.evaluate(validation_generator)
print(f'Loss: {loss}, MAE: {mae}')


typical_age_range = 80 
accuracy_percentage = (1 - mae / typical_age_range) * 100
print(f'Acuratețea estimată ca procentaj al intervalului tipic de vârstă: {accuracy_percentage:.2f}%')


model.save('final_resnet_model.h5')
print("Modelul a fost salvat cu succes.")


def load_and_prepare_image(image_path):
    img = load_img(image_path, target_size=(200, 200), color_mode='rgb')
    img = img_to_array(img)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img


img = load_and_prepare_image('woman.png')
predicted_age = model.predict(img)
print(f"Vârsta estimată este: {predicted_age[0][0]} ani")


100%|████████████████████████████████████████████████████████████████████████| 23708/23708 [00:00<00:00, 270542.83it/s]


Found 18967 validated image filenames.
Found 4741 validated image filenames.
Epoch 1/10


C:\Users\cosmi\OneDrive\Desktop\an4\licenta\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


593/593 ━━━━━━━━━━━━━━━━━━━━ 1699s 3s/step - loss: 509.0318 - mae: 17.3100 - val_loss: 382.8159 - val_mae: 15.1002
Epoch 2/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 1657s 3s/step - loss: 399.5399 - mae: 15.4290 - val_loss: 378.8488 - val_mae: 15.1555
Epoch 3/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 1643s 3s/step - loss: 389.4532 - mae: 15.1939 - val_loss: 376.7659 - val_mae: 14.8148
Epoch 4/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 1700s 3s/step - loss: 386.8159 - mae: 15.1483 - val_loss: 385.9794 - val_mae: 15.8888
Epoch 5/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 1731s 3s/step - loss: 397.5857 - mae: 15.4364 - val_loss: 366.7676 - val_mae: 14.9996
Epoch 6/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 1680s 3s/step - loss: 372.7231 - mae: 14.8874 - val_loss: 368.3130 - val_mae: 15.4501
Epoch 7/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 1644s 3s/step - loss: 372.2625 - mae: 14.8690 - val_loss: 354.0509 - val_mae: 14.5498
Epoch 8/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 1640s 3s/step - loss: 364.8647 - mae: 14.6981 - val_loss: 346.1296 - val_mae: 14.0571
Epo

Loss: 322.9332275390625, MAE: 13.957143783569336
Acuratețea estimată ca procentaj al intervalului tipic de vârstă: 82.55%
Modelul a fost salvat cu succes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Vârsta estimată este: 37.35746383666992 ani
